In [1]:
## classique 

import numpy as np
import torch 
import h5py # pour gérer les formats de données utilisés ici 
import torch
from collections import Counter
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

## handmade
import train
import test

On commence par charger les données d'entraînement, que l'on va plus tard utiliser pour entraîner le modèle de few-shot learning en question. 

In [ ]:
# Chemin vers le fichier de données
x, y, snr = test.load_data(type="enroll")


### Few-Shot Classification

In few-shot classification we are given a small support set of $ N $ labeled examples $ S = \{(x_1, y_1), \dots, (x_N, y_N)\} $, where each $ x_i \in \mathbb{R}^D $ is the $ D $-dimensional feature vector of an example and $ y_i \in \{1, \dots, K\} $ is the corresponding label. $ S_k $ denotes the set of examples labeled with class $ k $.

### Prototypical Networks classification

Prototypical networks compute an $ M $-dimensional representation $ c_k \in \mathbb{R}^M $, or prototype, of each class through an embedding function $ f_\phi : \mathbb{R}^D \to \mathbb{R}^M $ with learnable parameters $ \phi $. Each prototype is the mean vector of the embedded support points belonging to its class:

$$
c_k = \frac{1}{|S_k|} \sum_{(x_i, y_i) \in S_k} f_\phi(x_i) \tag{1}
$$

Given a distance function $ d : \mathbb{R}^M \times \mathbb{R}^M \to [0, +\infty) $, prototypical networks produce a distribution over classes for a query point $ x $ based on a softmax over distances to the prototypes in the embedding space:

$$
p_\phi(y = k \mid x) = \frac{\exp(-d(f_\phi(x), c_k))}{\sum_{k'} \exp(-d(f_\phi(x), c_{k'}))} \tag{2}
$$

Learning proceeds by minimizing the negative log-probability:

$$
J(\phi) = -\log p_\phi(y = k \mid x)
$$

of the true class $ k $ via stochastic gradient descent (SGD). Training episodes are formed by randomly selecting a subset of classes from the training set, then choosing a subset of examples within each class to act as the support set and a subset of the remainder to serve as query points. 


In [4]:
opt = {
    'train_dataset_path': 'train.hdf5',  # Path to the dataset
    'epochs': 50,  # Number of epochs for training
    'learning_rate': 0.001,  # Learning rate for the optimizer
    'lr_scheduler_step': 20,  # Step size for the learning rate scheduler
    'lr_scheduler_gamma': 0.5,  # Multiplicative factor for learning rate decay
    'iterations': 100,  # Number of episodes per epoch
    'classes_per_it_tr': 5,  # Number of classes per training iteration
    'num_support_tr': 20,  # Number of support samples per class for training
    'num_query_tr': 20,  # Number of query samples per class for training
    'classes_per_it_val': 5,  # Number of classes per validation iteration
    'num_support_val': 5,  # Number of support samples per class for validation
    'num_query_val': 15,  # Number of query samples per class for validation
    'manual_seed': 7,  # Manual seed for reproducibility
    'device': 'cuda:0'  # Whether to use CUDA for computation
}

In [5]:
model, best_acc, train_loss, train_acc = train.main(opt)

=== Epoch: 0 ===


100%|██████████| 100/100 [00:37<00:00,  2.70it/s]


Avg Train Loss: 1.1148308420181274, Avg Train Acc: 0.4220999984443188
=== Epoch: 1 ===


100%|██████████| 100/100 [00:37<00:00,  2.65it/s]


Avg Train Loss: 0.7292481771111489, Avg Train Acc: 0.6186999994516372
=== Epoch: 2 ===


...

Avg Train Loss: 0.35193532906472685, Avg Train Acc: 0.7949000018835067
=== Epoch: 48 ===


100%|██████████| 100/100 [00:55<00:00,  1.81it/s]


Avg Train Loss: 0.34706170186400415, Avg Train Acc: 0.8003999996185303
=== Epoch: 49 ===


100%|██████████| 100/100 [00:45<00:00,  2.21it/s]


Avg Train Loss: 0.3588626051694155, Avg Train Acc: 0.7949000006914139
Final model saved to /home/onyxia/work/PrototypicalFewShots-4/last_model.pth
